<a href="https://colab.research.google.com/github/yuhong-Peng/Banking-Analytics/blob/master/Banking_Analytic_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as nu
Bond=pd.read_excel(r'C:\Users\彭宇鸿\Desktop\Dataset.xlsx',sheet_name='Bond')
Mortgage=pd.read_excel(r'C:\Users\彭宇鸿\Desktop\Dataset.xlsx',sheet_name='Mortgage')
RetailLoan=pd.read_excel(r'C:\Users\彭宇鸿\Desktop\Dataset.xlsx',sheet_name='RetailLoan')

##Question1

Considering the Risk free rate changes during the period, so the present value of the coupon should be discounted back with different risk free rate.

In [0]:
#interest rate from the 9-25
#BondPrice=[]
#y=[0.0159,0.0146,0.0140,0.0154]
#length=len(Bond.id)
#i=0
#while i<length:
    #if (Bond.Time2Maturity[i]>10):
    #    yieldreturn=y[3]
    #elif(Bond.Time2Maturity[i]>5)and(Bond.Time2Maturity[i]<=10):
     #   yieldreturn=y[2]
    #elif(Bond.Time2Maturity[i]>3)and(Bond.Time2Maturity[i]<=5):
    #    yieldreturn=y[1]
    #else:
    #    yieldreturn=y[0]
    #if Bond.CouponsRemaining[i]==0:
    #    payyearly=0
    #    r=yieldreturn
    #    t=Bond.Time2Maturity[i]
    #else:
    #    payyearly=Bond.Time2Maturity[i]/Bond.CouponsRemaining[i]#coupon payment per year
    #    r=(1+yieldreturn)**(payyearly)-1 #effective interest rate
    #    t=Bond.CouponsRemaining[i]#time  remaining
    #couponpay=Bond.Principal[i]*Bond.YearlyRate[i]*0.01*payyearly#coupon payment per time 
    #bondprice=couponpay*(1/r-1/(r*(1+r)**t))+Bond.Principal[i]/(1+r)**t#formula to calculate the bond price
    #BondPrice.append(bondprice)
    #i=1+i

In [0]:
#interest rate from the 9-25
Princepledis=[]
y=[0.0159,0.0146,0.0140,0.0154]
length=len(Bond.id)
i=0
while i<length:
    if (Bond.Time2Maturity[i]>10):
        yieldreturn=y[3]
    elif(Bond.Time2Maturity[i]>5)and(Bond.Time2Maturity[i]<=10):
        yieldreturn=y[2]
    elif(Bond.Time2Maturity[i]>3)and(Bond.Time2Maturity[i]<=5):
        yieldreturn=y[1]
    else:
        yieldreturn=y[0]
    #formula to calculate the discounted bond principle
    bondprice1=(Bond.Principal[i]/(1+yieldreturn)**Bond.Time2Maturity[i])
    Princepledis.append(bondprice1)
    i=i+1

calculate the discounted coupon with varied interest rate

In [0]:
coupondis=[]
y=[0.0159,0.0146,0.0140,0.0154]
length=len(Bond.id)
i=0
cou=0
while i<length:
  #formula to calculate the present value of coupon
    if Bond.CouponsRemaining[i]==0:
        payyearly=1
        factor=0
        t=Bond.Time2Maturity[i]
    else:
        payyearly=Bond.Time2Maturity[i]/Bond.CouponsRemaining[i]#coupon payment per year
        t=Bond.CouponsRemaining[i]#time  remaining
        factor=1
    couponpay=Bond.Principal[i]*Bond.YearlyRate[i]*0.01*payyearly*factor
    rten=(1+y[3])**(payyearly)-1
    rfive=(1+y[2])**(payyearly)-1
    rthree=(1+y[1])**(payyearly)-1
    rone=(1+y[0])**(payyearly)-1
    t1=10//payyearly
    t2=5//payyearly
    t3=3//payyearly
    if (Bond.Time2Maturity[i]>10):
        cou=couponpay*((1-(1+rten)**(t1-t))/rten)*(1+rten)**(-t1)+couponpay*((1-(1+rfive)**(t2-t1))/rfive)*(1+rfive)**(-t2)+couponpay*((1-(1+rthree)**(t3-t2))/rthree)*(1+rthree)**(-t3)+couponpay*((1-(1+rone)**(-t3))/rone)
    elif(Bond.Time2Maturity[i]>5)and(Bond.Time2Maturity[i]<=10):
        cou=couponpay*((1-(1+rfive)**(t2-t1))/rfive)*(1+rfive)**(-t2)+couponpay*((1-(1+rthree)**(t3-t2))/rthree)*(1+rthree)**(-t3)+couponpay*((1-(1+rone)**(-t3))/rone)
    elif(Bond.Time2Maturity[i]>3)and(Bond.Time2Maturity[i]<=5):
        cou=couponpay*((1-(1+rthree)**(t3-t2))/rthree)*(1+rthree)**(-t3)+couponpay*((1-(1+rone)**(-t3))/rone)
    else:
        cou=couponpay*((1-(1+rone)**(-t3))/rone)
    coupondis.append(cou)
    i=i+1


In [0]:
BondPrice=coupondis+Princepledis

In [0]:
#TotalBondprice
TotalBondprice=sum(BondPrice*Bond.BondsHeld)/100
print(TotalBondprice)

##Question2 

In [0]:
#question 2
#define functions to calculate the capital K for each instrument
def capital_requirement_retail(PD, LGD):
    import numpy as np
    from scipy.stats import norm
    R =  0.03 * ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) 
    R += 0.16 * (1 - ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) )
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD
    return(K) 
import math
def capital_requirement_Bond(PD, LGD):
    import numpy as np
    from scipy.stats import norm
    R =  0.12 * ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) 
    R += 0.24 * (1 - ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) )
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD
    return(K) 
def capital_requirement_Mortgage(PD, LGD):
    import numpy as np
    from scipy.stats import norm
    R=0.15
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD
    return(K) 

In [0]:
#loop to get each K for bond
j=0
KBond=[]
BondPD=[]
while j<178502:
    if Bond.PD[j]<=0.0003:
        PD=0.0003
    else:
        PD=Bond.PD[j]
    LGD=0.75
    M=(1+(Bond.Time2Maturity[j]-2.5)*(0.11852-0.05478*nu.log(PD))**2)/(1-1.5*(0.11852-0.05478*nu.log(PD))**2)
    Kbond=capital_requirement_Bond(PD, LGD)*M
    KBond.append(Kbond)
    BondPD.append(PD)
    j=j+1

In [0]:
#expected loss of bond
BondEAD=BondPrice*Bond.BondsHeld
ELBond=sum(BondEAD*BondPD)*0.75
print(ELBond)
#capital  requirement of Bond
cpBond=sum(BondEAD*KBond)*12.5*0.115
print(cpBond)

In [0]:
print(KBond[0:10])

In [0]:
#loop to calculate K for Mortgage
j=0
KMortgage=[]
MortgagePD=[]
MortgageLGD=[]
while j<=28462:
    if Mortgage.PD[j]<0.0003:
        PD=0.0003
    else:
        PD=Mortgage.PD[j]
    if Mortgage.LGD[j]<0.05:
        LGD=0.05
    else:
        LGD=Mortgage.LGD[j]
    Kmortgage=capital_requirement_Mortgage(PD, LGD)
    KMortgage.append(Kmortgage)
    MortgagePD.append(PD)
    MortgageLGD.append(LGD)
    j=j+1


In [0]:
#EAD of Mortgage
EADMortgage=[]
j=0
while j<=28462:
    r=(1+Mortgage.Rate[j])**(1/12)-1
    Mpay=(r*Mortgage.OriginalAmount[j])/(1-(1+r)**(-Mortgage.TermYears[j]*12))
    Balance=Mpay*(1-(1+r)**(Mortgage.CurrentInstallment[j]-Mortgage.TermYears[j]*12))/r
    if Balance>Mortgage.Collateral_valuation[j]:
        EAD=Balance-Mortgage.Collateral_valuation[j]
    else:
        EAD=0
    EADMortgage.append(EAD)
    j=j+1


In [0]:
print(EADMortgage[0:10])
print(sum(EADMortgage))

In [0]:
#expected loss of Mortgage
j=0
ELMortgage=[]
while j<28462:
    ELM=EADMortgage[j]*MortgagePD[j]*MortgageLGD[j]
    ELMortgage.append(ELM)
    j=j+1
print(sum(ELMortgage))
#capital requirement of Mortgage
j=0
cpMortgage=[]
while j<28462:
    cpM=EADMortgage[j]*KMortgage[j]*12.5*0.115
    cpMortgage.append(cpM)
    j=j+1
print(sum(cpMortgage))

In [0]:
#loop to calculate K for RetailLoan
j=0
KRetailLoan=[]
RetailLoanPD=[]
while j<=21536:
    if RetailLoan.PD[j]<0.0003:
        PD=0.0003
    else:
        PD=RetailLoan.PD[j]
    LGD=1
    KRetailloan=capital_requirement_retail(PD, LGD)
    KRetailLoan.append(KRetailloan)
    RetailLoanPD.append(PD)
    j=j+1

In [0]:
print(RetailLoanPD[0:10])

In [0]:
#EAD of Loan
j=0
EADLoan=[]
while j<=21536:
    r=(1+RetailLoan.YearlyInterestRate[j])**(1/12)-1
    Mpay=(r*RetailLoan.OriginalAmount[j])/(1-(1+r)**(-RetailLoan.Term[j]))
    Balance=Mpay*(1-(1+r)**(RetailLoan.CurrentInstallment[j]-RetailLoan.Term[j]))/r
    EADLoan.append(Balance)
    j=j+1

In [0]:
print(sum(EADLoan))

In [0]:
#Expected loss of RetailLoan
j=0
ELRetailLoan=[]
while j<=21536:
    EL=RetailLoanPD[j]*EADLoan[j]
    ELRetailLoan.append(EL)
    j=j+1
print(sum(ELRetailLoan))
#capital requirement of RetailLoan
j=0
cpRetailLoan=[]
while j<=21536:
    cp=KRetailLoan[j]*EADLoan[j]*12.5*0.115
    cpRetailLoan.append(cp)
    j=j+1
print(sum(cpRetailLoan))
print(sum(EADLoan))

##Question 3

Risk Weighted assets for Bond

In [0]:
#question 3 to get the RWA 
#first for Bond
j=0
RwaBond=[]
while j<length:
    if Bond.PD[j]<=0.01:
        Rwa=0.2
    elif Bond.PD[j]>0.01 and Bond.PD[j]<=0.12:
        Rwa=0.5
    elif Bond.PD[j]>0.12 and Bond.PD[j]<=0.52:
        Rwa=1
    else:
        Rwa=1.5
    RwaBond.append(Rwa)
    j=j+1


In [0]:
#Risk wieghted asset
RWABond=sum(BondEAD*RwaBond)
print(RWABond)
#regulatory
j=0
RegBond=[]
while j<length:
    RegB=BondEAD[j]*RwaBond[j]*0.115
    RegBond.append(RegB)
    j=j+1
print(sum(RegBond))

Risk Weighted assets for Mortgage

In [0]:
#second for the Mortgage
j=0
RwaMortgage=[]
while j<=28462:
    if Mortgage.OriginalAmount[j]/Mortgage.CurrentInstallment[j]<0.8:
        rwamortgage=0.35
    else:
        rwamortgage=0.75
    RwaMortgage.append(rwamortgage)
    j=j+1
#useless for later question

In [0]:
print(RwaMortgage[0:10])

In [0]:
#RIsk wieghted asset 
j=0
RWAMortgage=[]
while j<28462:
    RWAM=EADMortgage[j]*0.35
    RWAMortgage.append(RWAM)
    j=j+1
print(sum(RWAMortgage))
#Regulatory of Mortgage
j=0
RMortgage=[]
while j<28462:
    RM=EADMortgage[j]*0.35*0.115
    RMortgage.append(RM)
    j=j+1
print(sum(RMortgage))

Risk Weighted asset for Rential Loan

In [0]:
RwaRetailLoan=0.75

In [0]:
#RWA of RetailLoan
j=0
RwaRetailLoan=[]
while j<=21536:
    RwaR=0.75*EADLoan[j]
    RwaRetailLoan.append(RwaR)
    j=j+1
print(sum(RwaRetailLoan))

In [0]:
#Capital requirement of RetailLoan
i=0
RwaRetailLoan=[]
while i<=21536:
    RwaR=0.75*EADLoan[i]*0.115
    RwaRetailLoan.append(RwaR)
    i=i+1
print(sum(RwaRetailLoan))